<a href="https://colab.research.google.com/github/stilldeserveahappyending/practice25/blob/main/age_predictor_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Производственная

Импорт библиотек

In [ ]:
!pip uninstall -y mtcnn lz4 joblib
!pip install --upgrade --force-reinstall mtcnn lz4 joblib opencv-python-headless

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

from mtcnn import MTCNN
import lz4

import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor
from google.colab import drive
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, recall_score
import joblib
import contextlib

drive.mount('/content/drive')


Датасет и эмбендинговая модель

In [ ]:
data_dir = '/content/drive/MyDrive/dataset/UTKFfix/UTKFace'
# Загрузка модели
model = tf.saved_model.load('/content/drive/MyDrive/dataset/faceNet')
infer = model.signatures['serving_default']

Обработка

In [ ]:
def preprocess_image(img_path):
    """Загрузка и предобработка изображения"""
    img = cv2.imread(img_path)
    if img is None:
        print(f"Не удалось загрузить изображение: {img_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (160, 160))
    img = img.astype(np.float32)
    img = (img - 127.5) / 128.0  # Нормализация для FaceNet
    return img

# Списки для данных
embeddings = []
ages = []

Получение эмбендингов

In [ ]:
def get_face_embedding(img_path):
    """Извлечение эмбеддинга лица"""
    img = preprocess_image(img_path)
    if img is None:
        return None

    img_array = np.expand_dims(img, axis=0)
    img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)

    # Получение эмбеддинга
    result = infer(img_tensor)
    embedding = result['Bottleneck_BatchNorm'].numpy()
    return embedding.squeeze(0)
# Обработка изображений
for img_name in os.listdir(data_dir):
    try:
        # Парсинг возраста из имени файла
        parts = img_name.split('_')
        if not parts[0].isdigit():
            continue

        age = int(parts[0])
        img_path = os.path.join(data_dir, img_name)

        # Генерация эмбеддинга
        embedding = get_face_embedding(img_path)
        if embedding is None:
            continue

        # Сохраняем данные
        embeddings.append(embedding)
        ages.append(age)

    except Exception as e:
        print(f"Ошибка обработки {img_name}: {str(e)}")
        continue

Обработка результатов

In [ ]:
# Преобразуем в numpy arrays
loaded_data = np.load('/content/drive/MyDrive/dataset/face_data.npz')
embeddings = loaded_data['embeddings']
ages = loaded_data['ages']
#embeddings = np.array(embeddings)
#ages = np.array(ages)

print("\nСтатистика датасета:")
print(f"Всего образцов: {len(embeddings)}")
print(f"Размерность эмбеддингов: {embeddings.shape}")
print(f"Минимальный возраст: {np.min(ages)}")
print(f"Максимальный возраст: {np.max(ages)}")
print(f"Средний возраст: {np.mean(ages):.1f}")

# Разделение данных без стратификации по группам
X_train, X_test, y_train, y_test = train_test_split(
    embeddings,
    ages,
    test_size=0.2,
    random_state=42
)

print(f"\nРазмер обучающей выборки: {X_train.shape[0]}")
print(f"Размер тестовой выборки: {X_test.shape[0]}")

Сохранение выборки

In [ ]:
save_dir = '/content/drive/MyDrive/age_prediction_data'
os.makedirs(save_dir, exist_ok=True)

# Сохраняем данные
np.save(os.path.join(save_dir, 'X_train.npy'), X_train)
np.save(os.path.join(save_dir, 'X_test.npy'), X_test)
np.save(os.path.join(save_dir, 'y_train.npy'), y_train)
np.save(os.path.join(save_dir, 'y_test.npy'), y_test)


Загрузка выборки

In [ ]:
load_dir = '/content/drive/MyDrive/age_prediction_data'

# Загрузка данных
X_train = np.load(os.path.join(load_dir, 'X_train.npy'))
X_test = np.load(os.path.join(load_dir, 'X_test.npy'))
y_train = np.load(os.path.join(load_dir, 'y_train.npy'))
y_test = np.load(os.path.join(load_dir, 'y_test.npy'))

MTCNN

In [ ]:
img = cv2.imread('/content/drive/MyDrive/dataset/train/18-20/5.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
face_detector = MTCNN()
detections = face_detector.detect_faces(img)

# Лицо с наибольшей уверенностью
best_face = max(detections, key=lambda x: x['confidence'])
x, y, w, h = best_face['box']

# Увеличивание области вокруг лица на 10%
margin = 0.1
x = max(0, int(x - margin * w))
y = max(0, int(y - margin * h))
w = min(img.shape[1] - x, int(w * (1 + 2 * margin)))
h = min(img.shape[0] - y, int(h * (1 + 2 * margin)))

# Обрезание лица
face = img[y:y+h, x:x+w]
plt.figure(figsize=(10, 8))

# Отрисовка прямоугольника вокруг лица
rect = plt.Rectangle((x, y), w, h, fill=False, color='red', linewidth=2)

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.gca().add_patch(rect)
plt.title('Исходное изображение с обнаруженным лицом')
plt.axis('off')

# Вывод
plt.subplot(1, 2, 2)
plt.imshow(face)
plt.title('Обрезанная область лица')
plt.axis('off')

plt.tight_layout()
plt.show()

Модель предсказания возраста по эмбендингам

In [ ]:
class AgePredictor:
    def __init__(self, embedding_model_path):
        """Инициализация"""
        # Загрузка модели для получения эмбеддингов лиц
        self.embedding_model = tf.saved_model.load(embedding_model_path)
        self.embedder = self.embedding_model.signatures['serving_default']

        # Масштабирование признаков
        self.scaler = StandardScaler()

        # Ансамбль
        self.ensemble = None

        # Отдельные модели
        self.individual_models = {}

        # Метрики оценки
        self.evaluation_metrics = {}
        try:
            # Детектор лиц
            self.face_detector = MTCNN()
        except Exception as e:
            print(f"Ошибка инициализации MTCNN: {str(e)}")
            self.face_detector = None

    def cutting_face(self, img):
        """Обнаружение лица на изображении"""
        if self.face_detector is None:
            return img

        # Обнаружение лиц на изображении
        detections = self.face_detector.detect_faces(img)
        if not detections:
            return None

        # Лицо с наибольшей уверенностью
        best_face = max(detections, key=lambda x: x['confidence'])
        x, y, w, h = best_face['box']

        # Отступ 10%
        margin = 0.1

        # Корректировка отступа
        x = max(0, int(x - margin * w))
        y = max(0, int(y - margin * h))
        w = min(img.shape[1] - x, int(w * (1 + 2 * margin)))
        h = min(img.shape[0] - y, int(h * (1 + 2 * margin)))

        return img[y:y+h, x:x+w]

    def preprocess_image(self, img_path):
        """Загрузка и предобработка изображения"""
        # Загрузка изображения
        img = cv2.imread(img_path)
        if img is None:
            print("Не удалось загрузить изображение")
            return None

        # Перевод в необходимый формат для работы MTCNN
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Обнаружение и обрезка лица
        face = self.cutting_face(img)
        if face is None:
            print("Лицо не обнаружено")
            return None

        # Изененние размера и перевод в float32 (требования FaceNet)
        face = cv2.resize(face, (160, 160))
        face = face.astype(np.float32)

        # Нормализация
        face = (face - 127.5) / 128.0

        return face

    def predict_age_from_image(self, img_path):
        """Предсказание возраста"""
        # Получение эмбеддинга
        embedding = self.get_face_embedding(img_path)
        if embedding is None:
            return None

        # Масштабирование
        scaled_embedding = self.scaler.transform([embedding])

        # Предсказание возраста
        age = self.ensemble.predict(scaled_embedding)[0]

        # Возврат с округлением до 1 знака
        return round(age, 1)

    def get_face_embedding(self, img_path):
        """Получение эмбеддинга лица"""
        img = self.preprocess_image(img_path)
        if img is None:
            return None

        # Расширение измерений (требования FaceNet, форма фходных данных: (1, 160, 160, 3))
        img_array = np.expand_dims(img, axis=0)
        img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)

        # Получение эмбеддинга
        result = self.embedder(img_tensor)
        embedding = result['Bottleneck_BatchNorm'].numpy()
        return embedding.squeeze(0)

    def train_models(self, X, y):
        """Обучение модели"""
        # Масштабирование данных
        X_scaled = self.scaler.fit_transform(X)

        # Обучение SVR с RBF ядром
        svm_rbf = SVR(kernel='rbf', gamma='auto', epsilon=0.05, C=100)
        svm_rbf.fit(X_scaled, y)
        self.individual_models['svm_rbf'] = svm_rbf

        # Обучение SVR с полиномиальным ядром
        svm_poly = SVR(kernel='poly', epsilon=0.1, degree=2, coef0=0.5, C=100)
        svm_poly.fit(X_scaled, y)
        self.individual_models['svm_poly'] = svm_poly

        # Создание ансамбля
        self.ensemble = VotingRegressor(
            estimators=[
                ('svm_rbf', svm_rbf),
                ('svm_poly', svm_poly)
            ],
            weights=[2, 1]  # Вес модели с RBF ядром в 2 раза больше
        )
        self.ensemble.fit(X_scaled, y)

        return self

    def evaluate_model(self, X_test, y_test):
        """Оценка модели"""
        if self.ensemble is None:
            print("Модель не обучена")
            return None

        # Масштабирование тестовых данных
        X_test_scaled = self.scaler.transform(X_test)

        # Предсказание на тестовых данных
        y_pred = self.ensemble.predict(X_test_scaled)

        # Вычисление метрик
        metrics = {
            'MSE': mean_squared_error(y_test, y_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
            'R2': r2_score(y_test, y_pred),
            'MAE': np.mean(np.abs((y_test - y_pred)))
        }

        # Сохранение метрик
        self.evaluation_metrics = {
            'ensemble': metrics
        }

        return self.evaluation_metrics

    def print_metrics(self):
        """Вывод метрик оценки"""
        if not self.evaluation_metrics:
            return

        print("\nМетрики оценки модели:")
        for metric, value in self.evaluation_metrics['ensemble'].items():
            print(f"{metric}: {value:.4f}")

    def save_models(self, path):
        """Сохранение модели"""
        if self.ensemble is None:
            return
        joblib.dump({
            'ensemble': self.ensemble,
            'individual_models': self.individual_models,
            'scaler': self.scaler
        }, path, compress=('gzip', 3))

    @classmethod
    def load_models(cls, embedding_model_path, predictor_path):
        """Загрузка модели"""
        # Создание экземпляра класса
        predictor = cls(embedding_model_path)

        # Загрузка сохраненной модели
        models = joblib.load(predictor_path)
        predictor.ensemble = models['ensemble']
        predictor.individual_models = models['individual_models']
        predictor.scaler = models['scaler']

        return predictor

Результат

In [ ]:
# Основной блок выполнения
if __name__ == "__main__":
    # Инициализация
    predictor = AgePredictor('/content/drive/MyDrive/dataset/faceNet')
    predictor.train_models(X_train, y_train)

    # Оценка модели
    print("\nОценка модели на тестовых данных...")
    metrics = predictor.evaluate_model(X_test, y_test)
    predictor.print_metrics()

    model_dir = '/content/drive/MyDrive/age_detection_models'
    os.makedirs(model_dir, exist_ok=True)

    predictor_path = os.path.join(model_dir, 'SVR_age_predictor_fix.pkl')
    predictor.save_models(predictor_path)

Загрузка модели

In [ ]:
model_path = '/content/drive/MyDrive/age_detection_models/Age_predictor_SVR_fin.pkl'
faceNet_path = '/content/drive/MyDrive/dataset/faceNet'
predictor = AgePredictor.load_models(
    embedding_model_path=faceNet_path,
    predictor_path=model_path
)


Использование

In [ ]:
if __name__ == "__main__":
    # Тестирование на новом изображении
    test_img = '/content/drive/MyDrive/dataset/train/51-60/13.jpg'
    result = predictor.predict_age_from_image(test_img)
    if result:
        img = cv2.imread(test_img)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(8, 6))
        plt.imshow(img_rgb)
        plt.title(f"Предсказанный возраст: {result} лет")
        plt.axis('off')
        plt.show()